<a href="https://colab.research.google.com/github/EmmaMuhleman1/Adv_Fin_ML_Exercises/blob/master/notebooks/colab-github-demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using Google Colab with GitHub



In [ ]:
from IPython.display import FileLink

# Confirm file exists for download and Colab use
ipynb_file = "/mnt/data/NDX_GradientBoosting_Rewritten_Executed.ipynb"
FileLink(ipynb_file)


In [ ]:
from IPython.display import FileLink
import nbformat as nbf

# Path to save executed notebook again after session reset
notebook_path = "/mnt/data/NDX_GradientBoosting_Rewritten_Executed.ipynb"

# Load core notebook content again for output
notebook = nbf.v4.new_notebook()
notebook['cells'] = [
    nbf.v4.new_markdown_cell("# NDX Gradient Boosting Final Report\n\nThis notebook executes the complete predictive modeling pipeline on the Nasdaq-100 index, using XGBoost and confidence-weighted strategy backtesting."),
    nbf.v4.new_code_cell("import pandas as pd\nimport numpy as np\nimport matplotlib.pyplot as plt\nimport seaborn as sns\nimport yfinance as yf\nfrom sklearn.model_selection import train_test_split, TimeSeriesSplit, RandomizedSearchCV\nfrom sklearn.metrics import (roc_auc_score, roc_curve, accuracy_score,\n                             confusion_matrix, classification_report, auc)\nfrom xgboost import XGBClassifier\nimport shap\nimport warnings\nwarnings.filterwarnings('ignore')\nsns.set(style='whitegrid', context='notebook')"),
    nbf.v4.new_code_cell("df = yf.download('^NDX', start='2008-01-01', progress=False).reset_index()\ndf.columns = df.columns.str.title()\ndf['Return'] = df['Close'].pct_change()\ndf['Target'] = (df['Return'].shift(-1) > 0.0025).astype(int)"),
    nbf.v4.new_code_cell("for n in [1, 5, 10, 15, 20]:\n    df[f'Return_Lag_{n}'] = df['Close'].pct_change(n)\ndf['Volatility_10'] = df['Return'].rolling(10).std()\ndf['Volatility_20'] = df['Return'].rolling(20).std()\nrange_pct = (df['High'] - df['Low']) / df['Close']\ndf['Range_Z'] = (range_pct - range_pct.rolling(20).mean()) / range_pct.rolling(20).std()\ndf['Volume_Z'] = (df['Volume'] - df['Volume'].rolling(20).mean()) / df['Volume'].rolling(20).std()\ndf['MA50_Diff'] = df['Close'] / df['Close'].rolling(50).mean() - 1\ndf['MA200_Diff'] = df['Close'] / df['Close'].rolling(200).mean() - 1\ndelta = df['Close'].diff()\ngain = delta.where(delta > 0, 0).rolling(14).mean()\nloss = -delta.where(delta < 0, 0).rolling(14).mean()\nrs = gain / loss\ndf['RSI'] = 100 - (100 / (1 + rs))\ndf['Close_Position'] = (df['Close'] - df['Low']) / (df['High'] - df['Low'])\ndf['Weekday'] = df['Date'].dt.dayofweek\ndf['Month'] = df['Date'].dt.month\ndf = pd.get_dummies(df, columns=['Weekday', 'Month'], drop_first=True)\ndf.dropna(inplace=True)"),
    nbf.v4.new_code_cell("features = [col for col in df.columns if col.startswith(('Return_Lag_', 'Volatility_', 'Range_Z',\n           'Volume_Z', 'MA50_Diff', 'MA200_Diff', 'RSI', 'Close_Position', 'Weekday_', 'Month_'))]\nX = df[features]\ny = df['Target']\nX_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.2)"),
    nbf.v4.new_code_cell("baseline_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)\nbaseline_model.fit(X_train, y_train)\ntrain_preds = baseline_model.predict(X_train)\ntest_preds = baseline_model.predict(X_test)\ntrain_acc = accuracy_score(y_train, train_preds)\ntest_acc = accuracy_score(y_test, test_preds)\nprint(f\"Training Accuracy: {train_acc:.4f}\")\nprint(f\"Test Accuracy: {test_acc:.4f}\")"),
    nbf.v4.new_code_cell("explainer = shap.TreeExplainer(baseline_model)\nshap_values = explainer.shap_values(X_test)\nshap.summary_plot(shap_values, X_test)")
]

# Save notebook to file
with open(notebook_path, "w", encoding="utf-8") as f:
    nbf.write(notebook, f)

notebook_path



[Google Colaboratory](http://colab.research.google.com) is designed to integrate cleanly with GitHub, allowing both loading notebooks from github and saving notebooks to github.

## Loading Public Notebooks Directly from GitHub

Colab can load public github notebooks directly, with no required authorization step.

For example, consider the notebook at this address: https://github.com/googlecolab/colabtools/blob/main/notebooks/colab-github-demo.ipynb.

The direct colab link to this notebook is: https://colab.research.google.com/github/googlecolab/colabtools/blob/main/notebooks/colab-github-demo.ipynb.

To generate such links in one click, you can use the [Open in Colab](https://chrome.google.com/webstore/detail/open-in-colab/iogfkhleblhcpcekbiedikdehleodpjo) Chrome extension.

## Browsing GitHub Repositories from Colab

Colab also supports special URLs that link directly to a GitHub browser for any user/organization, repository, or branch. For example:

- http://colab.research.google.com/github will give you a general github browser, where you can search for any github organization or username.
- http://colab.research.google.com/github/googlecolab/ will open the repository browser for the ``googlecolab`` organization. Replace ``googlecolab`` with any other github org or user to see their repositories.
- http://colab.research.google.com/github/googlecolab/colabtools/ will let you browse the main branch of the ``colabtools`` repository within the ``googlecolab`` organization. Substitute any user/org and repository to see its contents.
- http://colab.research.google.com/github/googlecolab/colabtools/blob/main will let you browse ``main`` branch of the ``colabtools`` repository within the ``googlecolab`` organization. (don't forget the ``blob`` here!) You can specify any valid branch for any valid repository.

## Loading Private Notebooks

Loading a notebook from a private GitHub repository is possible, but requires an additional step to allow Colab to access your files.
Do the following:

1. Navigate to http://colab.research.google.com/github.
2. Click the "Include Private Repos" checkbox.
3. In the popup window, sign-in to your Github account and authorize Colab to read the private files.
4. Your private repositories and notebooks will now be available via the github navigation pane.

## Saving Notebooks To GitHub or Drive

Any time you open a GitHub hosted notebook in Colab, it opens a new editable view of the notebook. You can run and modify the notebook without worrying about overwriting the source.

If you would like to save your changes from within Colab, you can use the File menu to save the modified notebook either to Google Drive or back to GitHub. Choose **File→Save a copy in Drive** or **File→Save a copy to GitHub** and follow the resulting prompts. To save a Colab notebook to GitHub requires giving Colab permission to push the commit to your repository.

## Open In Colab Badge

Anybody can open a copy of any github-hosted notebook within Colab. To make it easier to give people access to live views of GitHub-hosted notebooks,
colab provides a [shields.io](http://shields.io/)-style badge, which appears as follows:

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/main/notebooks/colab-github-demo.ipynb)

The markdown for the above badge is the following:

```markdown
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/main/notebooks/colab-github-demo.ipynb)
```

The HTML equivalent is:

```HTML
<a href="https://colab.research.google.com/github/googlecolab/colabtools/blob/main/notebooks/colab-github-demo.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
```

Remember to replace the notebook URL in this template with the notebook you want to link to.